### Importing the libraries

In [2]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SearchIndex
)
from util import *

### Loading the product images

In [ ]:
EMBEDDINGS_DIR = "embeddings"

os.makedirs(EMBEDDINGS_DIR, exist_ok=True)
image_directory = os.path.join('images')
embedding_directory = os.path.join('embeddings')
output_json_file = os.path.join(embedding_directory, 'output.jsonl')

### Create a data source

In [ ]:
ds_client = SearchIndexerClient(acs_endpoint, AzureKeyCredential(acs_key))
container = SearchIndexerDataContainer(name=container_name)
data_source_connection = SearchIndexerDataSourceConnection(
    name=f"{index_name}-blob",
    type="azureblob",
    connection_string=blob_connection_string,
    container=container,
)
data_source = ds_client.create_or_update_data_source_connection(data_source_connection)

print(f"Done. Data source '{data_source.name}' has been created or updated.")

In [ ]:
# Connect to Blob Storage
blob_service_client = BlobServiceClient.from_connection_string(blob_connection_string)
container_client = blob_service_client.get_container_client(container_name)
blobs = container_client.list_blobs()

first_blob = next(blobs)
blob_url = container_client.get_blob_client(first_blob).url
print(f"URL of the first blob: {blob_url}")

### Create a embedding json using Cognitive Vision API

In [ ]:
with open(output_json_file, 'w') as outfile:
    for idx, image_path in enumerate(os.listdir(image_directory)):
        if image_path:
            try:
                vector = image_embedding(os.path.join(image_directory, image_path))
            except Exception as e:
                print(f"Error processing image at index {idx}: {e}")
                vector = None
            
            filename, _ = os.path.splitext(os.path.basename(image_path))
            result = {
                "id": f'{idx}',
                "image_vector": vector,
                "description": filename
            }

            outfile.write(json.dumps(result))
            outfile.write('\n')
            outfile.flush()

print(f"Results are saved to {output_json_file}")

In [ ]:
credential = AzureKeyCredential(acs_key)
# Create a search index 
index_client = SearchIndexClient(endpoint=acs_endpoint, credential=credential)  
fields = [  
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),  
    SearchField(name="description", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),  
    SearchField(
        name="image_vector",  
        hidden=True,
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single), 
        searchable=True,
        vector_search_dimensions=1024,  
        vector_search_profile_name="myHnswProfile"
    ),  
]

### Configure the vector search configuration  

In [ ]:
vector_search = VectorSearch(  
    algorithms=[  
        HnswAlgorithmConfiguration(  
            name="myHnsw"
        )
    ],  
   profiles=[  
        VectorSearchProfile(  
            name="myHnswProfile",  
            algorithm_configuration_name="myHnsw",
        )
    ],  
)  
  
# Create the search index with the vector search configuration  
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)  
result = index_client.create_or_update_index(index)  
print(f"{result.name} created")

### Ingest Embedding into Azure Search

In [ ]:
from azure.search.documents import SearchClient
import json

data = []
with open(output_json_file, 'r') as file:
    for line in file:
        # Remove leading/trailing whitespace and parse JSON
        json_data = json.loads(line.strip())
        data.append(json_data)

search_client = SearchClient(endpoint=acs_endpoint, index_name=index_name, credential=credential)
results = search_client.upload_documents(data)
for result in results:
    print(f'Indexed {result.key} with status code {result.status_code}')

### Upload the images to blob store to use it during retrival

In [ ]:
for root, dirs, files in os.walk(image_directory):
    for file in files:
        local_file_path = os.path.join(root, file)
        blob_name = os.path.relpath(local_file_path, image_directory)
        with open(local_file_path, "rb") as data:
            blob_client.upload_blob(data, overwrite=True)

## Importing Libraries

In [ ]:
from langchain_core.prompts.chat import (
    BaseMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    PromptTemplate,
)
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_community.tools.convert_to_openai import format_tool_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.output_parsers.openai_functions import (
    OpenAIFunctionsAgentOutputParser,
)
from langchain.agents.format_scratchpad.openai_functions import (
    format_to_openai_function_messages,
)
from langchain.memory import PostgresChatMessageHistory
from langchain.agents import AgentExecutor
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import RunnableConfig
from langchain.schema.messages import AIMessageChunk
import os
from uuid import uuid4
from typing import Optional

from custom_tool import ImageSearchResults
import openai

### Loading LangChain LLM

In [ ]:
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
    api_key=os.environ["AZURE_OPENAI_KEY"],
    api_version="2023-12-01-preview",
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    model="gpt-4-turbo",
)
prefix="""You are rambo a helpful Fashion Agent who help people navigating and buying products online

Note:

\\ Show Prices always in INR
\\ Always try user to buy from the buy now link provided"""
suffix = ""

tools = [ImageSearchResults(num_results=5)]
llm_with_tools = llm.bind(
    functions=[convert_to_openai_function(t) for t in tools]
)
messages = [
    SystemMessage(content=prefix),
    HumanMessagePromptTemplate.from_template("{input}"),
    AIMessage(content=suffix),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
]
input_variables = ["input", "agent_scratchpad"]
prompt = ChatPromptTemplate(input_variables=input_variables, messages=messages)

### Creating an Agent

In [ ]:
agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        )
    )
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)
message_history = PostgresChatMessageHistory(
        connection_string=os.environ["POSTGRES_CONNECTION_STRING"],
        session_id= identifier + str(uuid4()),
    )
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

### Running Agent based on input Query

In [ ]:
config = RunnableConfig(callbacks = [cl.AsyncLangchainCallbackHandler()])
input = "Blue tshirt for summer"
async for patch in agent_executor.astream_log({"input": input, "chat_history": message_history.messages}, 
                                              config = RunnableConfig(callbacks = [cl.AsyncLangchainCallbackHandler(stream_final_answer = True)])):
    for op in patch.ops:
        if op["op"] != "add":
            continue
        value = op["value"]
        if not isinstance(value, AIMessageChunk):
            continue
        if value.content == "":
            continue
        await msg.stream_token(value.content)

### From a given input image search Product Catalog

In [ ]:
from openai import AzureOpenAI
client = AzureOpenAI()

### Describe Image

In [ ]:
describe_system_prompt = '''
    You are a system generating descriptions for furniture items, decorative items, or furnishings on an e-commerce website.
    Provided with an image and a title, you will describe the main item that you see in the image, giving details but staying concise.
    You can describe unambiguously what the item is and its material, color, and style if clearly identifiable.
    If there are multiple items depicted, refer to the title to understand which item you should describe.
    '''

def describe_image(img_url, title):
    response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    temperature=0.2,
    messages=[
        {
            "role": "system",
            "content": describe_system_prompt
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": img_url,
                },
            ],
        },
        {
            "role": "user",
            "content": title
        }
    ],
    max_tokens=300,
    )

    return response.choices[0].message.content

### Captioning Image

In [ ]:
caption_system_prompt = '''
Your goal is to generate short, descriptive captions for images of furniture items, decorative items, or furnishings based on an image description.
You will be provided with a description of an item image and you will output a caption that captures the most important information about the item.
Your generated caption should be short (1 sentence), and include the most relevant information about the item.
The most important information could be: the type of the item, the style (if mentioned), the material if especially relevant and any distinctive features.
'''

few_shot_examples = [
    {
        "description": "This is a multi-layer metal shoe rack featuring a free-standing design. It has a clean, white finish that gives it a modern and versatile look, suitable for various home decors. The rack includes several horizontal shelves dedicated to organizing shoes, providing ample space for multiple pairs. Above the shoe storage area, there are 8 double hooks arranged in two rows, offering additional functionality for hanging items such as hats, scarves, or bags. The overall structure is sleek and space-saving, making it an ideal choice for placement in living rooms, bathrooms, hallways, or entryways where efficient use of space is essential.",
        "caption": "White metal free-standing shoe rack"
    },
    {
        "description": "The image shows a set of two dining chairs in black. These chairs are upholstered in a leather-like material, giving them a sleek and sophisticated appearance. The design features straight lines with a slight curve at the top of the high backrest, which adds a touch of elegance. The chairs have a simple, vertical stitching detail on the backrest, providing a subtle decorative element. The legs are also black, creating a uniform look that would complement a contemporary dining room setting. The chairs appear to be designed for comfort and style, suitable for both casual and formal dining environments.",
        "caption": "Set of 2 modern black leather dining chairs"
    },
    {
        "description": "This is a square plant repotting mat designed for indoor gardening tasks such as transplanting and changing soil for plants. It measures 26.8 inches by 26.8 inches and is made from a waterproof material, which appears to be a durable, easy-to-clean fabric in a vibrant green color. The edges of the mat are raised with integrated corner loops, likely to keep soil and water contained during gardening activities. The mat is foldable, enhancing its portability, and can be used as a protective surface for various gardening projects, including working with succulents. It's a practical accessory for garden enthusiasts and makes for a thoughtful gift for those who enjoy indoor plant care.",
        "caption": "Waterproof square plant repotting mat"
    }
]

formatted_examples = [[{
    "role": "user",
    "content": ex['description']
},
{
    "role": "assistant", 
    "content": ex['caption']
}]
    for ex in few_shot_examples
]

formatted_examples = [i for ex in formatted_examples for i in ex]

def caption_image(description, model="gpt-4-turbo-preview"):
    messages = formatted_examples
    messages.insert(0, 
        {
            "role": "system",
            "content": caption_system_prompt
        })
    messages.append(
        {
            "role": "user",
            "content": description
        })
    response = client.chat.completions.create(
    model=model,
    temperature=0.2,
    messages=messages
    )

    return response.choices[0].message.content

In [ ]:
def tag_and_caption(primary_image):
    img_description = describe_image(primary_image)
    caption = caption_image(img_description)
    return {
        'img_description': img_description,
        'caption': caption
    }

### Search the Catalog Image from the above Caption

In [ ]:
config = RunnableConfig(callbacks = [cl.AsyncLangchainCallbackHandler()])
result = tag_and_caption("image.png")
async for patch in agent_executor.astream_log({"input": result["caption"], "chat_history": message_history.messages}, 
                                              config = RunnableConfig(callbacks = [cl.AsyncLangchainCallbackHandler(stream_final_answer = True)])):
    for op in patch.ops:
        if op["op"] != "add":
            continue
        value = op["value"]
        if not isinstance(value, AIMessageChunk):
            continue
        if value.content == "":
            continue
        await msg.stream_token(value.content)